In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.wcs import WCS
import numpy as np
from matplotlib.colors import LogNorm
import astropy.io.fits as fits
from reproject import reproject_from_healpix
import healpy as hp
from matplotlib import cm
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic
from astropy import units as u
import copy
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from mpl_point_clicker import clicker

## Read in data cube:

In [ ]:
####################################
##### User inputs: #################
#hdu = fits.open('/home/ordoga/DATA/CHIME/CHIME_Fall_2022/RM_SYNTH_v2/FDF_tot_dirty.fits')
#hdu = fits.open('/srv/data/chime/chime_FD_Oct2022/Q_600_800_Oct132022_fix.hpx.fits')
hdu = fits.open('/srv/data/chime/chime_FD_Oct2022/FDF_tot_dirty_Oct132022.hpx.fits')
####################################

hdr = hdu[0].header
data = hdu[0].data

spectr_idx = np.linspace(0,hdr['NAXIS2']-1,hdr['NAXIS2'])
spectr_ax = hdr['CRVAL2']+(spectr_idx - hdr['CRPIX2']+1)*hdr['CDELT2']

if hdr['COORDSYS'] == 'C':
    frame = ICRS()
if hdr['COORDSYS'] == 'G':
    frame = Galactic()
    
hpx = HEALPix(nside=hdr['NSIDE'], order=hdr['ORDERING'], frame=frame)

print(repr(hdr))

## Define functions:

In [ ]:
def platcar_map(data,coordsys,pixsize=0.5,fs=14,cmap=cm.viridis,title='',bg_clr='gray',
               xc=180,yc=0,grid_clr='black',lbl_clr='black',axtitles=True,cbarlbl='',
               *args, **kwargs):
    proj = '-CAR'
    fc = 'Rect'
        
    hdr_new,cs = make_new_header(coordsys,proj,pixsize,xc,yc)
    fig, ax, ax2 = make_the_map(data[0,:],hdr_new,fc,cs,fs,cmap,title,bg_clr,grid_clr,lbl_clr,axtitles,cbarlbl,proj)
        
    return fig, ax, ax2, hdr_new, cs

def make_new_header(coordsys,proj,pixsize,xc,yc):
    if coordsys == 'C':
        ctype1 = 'RA--'+proj
        ctype2 = 'DEC-'+proj
        cs = 'icrs'
    if coordsys == 'G':
        ctype1 = 'GLON'+proj
        ctype2 = 'GLAT'+proj
        cs = 'galactic'

    if (proj == '-MOL') or (proj == '-AIT'):
        nx = int(round((4*np.sqrt(2)*180/(pixsize*np.pi)),0))
        ny = int(round((2*np.sqrt(2)*180/(pixsize*np.pi)),0))
    if (proj == '-CAR'):
        nx = int(round((360/pixsize),0))
        ny = int(round((180/pixsize),0))
    if (proj == '-TAN'):
        nx = int(round((15*180/(pixsize*np.pi)),0))
        ny = int(round((15*180/(pixsize*np.pi)),0))

    hdr_new = fits.Header.fromstring("""
NAXIS   =                    2
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')
    hdr_new['NAXIS1']  = nx 
    hdr_new['NAXIS2']  = ny
    hdr_new['CTYPE1']  = ctype1
    hdr_new['CRPIX1']  = nx/2.+0.5 
    hdr_new['CRVAL1']  = xc          
    hdr_new['CDELT1']  = -pixsize 
    hdr_new['CTYPE2']  = ctype2
    hdr_new['CRPIX2']  = ny/2.+0.5
    hdr_new['CRVAL2']  = yc
    hdr_new['CDELT2']  = pixsize
    
    return(hdr_new,cs)


def make_the_map(data,hdr_new,fc,cs,fs,cmap,title,bg_clr,
                 grid_clr,lbl_clr,axtitles,cbarlbl,proj):

    array, footprint = reproject_from_healpix((data,cs),hdr_new, nested=False)
    
    fig = plt.figure(figsize=(10,5))
    
    if fc == 'EllipticalFrame':
        ax = plt.subplot(121, projection=WCS(hdr_new),frame_class=EllipticalFrame)
    else:
        ax = plt.subplot(121, projection=WCS(hdr_new))       
    ax2 = plt.subplot(122)
    
    ax.coords.grid(color=grid_clr)
    ax.coords[0].set_ticklabel(color=lbl_clr,fontsize=fs)
    ax.coords[1].set_ticklabel(color=lbl_clr,fontsize=fs)
    ax.set_title(title,fontsize=fs+2)
    if axtitles:
        if (proj == '-MOL') or (proj == '-AIT'):
            ypad = 15
        else:
            ypad = 1
        if cs == 'icrs':            
            ax.coords[0].set_axislabel('Right Ascension',fontsize=fs,minpad=ypad)
            ax.coords[1].set_axislabel('Declination',fontsize=fs)
        if cs == 'galactic':
            ax.coords[0].set_axislabel('Galactic Longitude',fontsize=fs,minpad=ypad)
            ax.coords[1].set_axislabel('Galactic Latitude',fontsize=fs)
    
    return fig, ax, ax2

def plot_spectrum(klicker,hdr_new,hpx,data,spectr_ax,ax2,spectr_min,spectr_max,ymin,ymax):

    w1 = WCS(hdr_new)
    clikpos = klicker.get_positions().get('s')[-1]
    sky = w1.pixel_to_world(clikpos[0], clikpos[1])
    RAklik = sky.ra.deg
    decklik = sky.dec.deg
    print(sky.ra.hour)
    print(decklik)
    coord = SkyCoord(RAklik*u.deg,decklik*u.deg)
    hpx_coord = hpx.skycoord_to_healpix(coord)
    
    ax2.plot(spectr_ax,data[:,hpx_coord],
             label='RA='+str(round(sky.ra.hour,1))+r'h, $\delta$='+str(round(decklik,1))+'$^{\circ}$')
    ax2.set_xlim(spectr_min,spectr_max)
    ax2.set_ylim(ymin,ymax)
    ax2.grid(visible=True)
    ax2.legend(fontsize=8)
             
    return

def call_mapper(vmin,vmax,cdelt,spectr_val):
    
    #print(np.where(spectr_ax == spectr_val)[0])
    i = np.where(abs(spectr_ax-spectr_val)<cdelt/2.)[0][0]
    
    print(i)
    if hdr['ORDERING'] == 'RING':
        nested = False
    else:
        nested = True
    array, footprint = reproject_from_healpix((data[i,:],cs),hdr_new, nested=nested)
    cmap = mpl.cm.get_cmap("viridis").copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(array, vmin=vmin, vmax=vmax,cmap=cmap)
    return spectr_val


## User-selected parameters:

In [ ]:
#########################################################
##### User inputs: ######################################
pixsize = 0.5     # Width of pixels to use in map
fs = 10           # Fontsize for plots
xc = 180          # Horizontal centre of map (degrees)
yc = 0            # Vertical centre of map (degrees)
vmin = 0          # Minimum power on colorscale
vmax = 1          # Maximum power on colorscale
ymin = -2.0       # Minimum power shown in spectrum
ymax = 2.0        # Maximum power shown in spectrum
#########################################################

## Run the interactive code
### NOTE: run the second cell below each time you need a spectrum from the last point clicked

In [ ]:
fig,ax,ax2, hdr_new,cs = platcar_map(data,'C',pixsize=pixsize,fs=fs,cmap=cm.viridis,title='',bg_clr='gray',
                           xc=xc,yc=yc,grid_clr='black',lbl_clr='black',axtitles=True,cbarlbl='')

interact(call_mapper, vmin=fixed(vmin),vmax=fixed(vmax),cdelt = fixed(abs(hdr['CDELT2'])),
         spectr_val=widgets.FloatSlider(min=np.min(spectr_ax), max=np.max(spectr_ax), 
                                      step=abs(hdr['CDELT2']), value=hdr['CRVAL2'], 
                                      continuous_update=False,layout=Layout(width='800px')))

klicker = clicker(ax, ["s"], markers=["x"])
ax.get_legend().remove()

In [ ]:
plot_spectrum(klicker,hdr_new,hpx,data,spectr_ax,ax2,np.min(spectr_ax),np.max(spectr_ax),ymin,ymax)
ax2.grid(True)